In the Neuron notebook we looked at the single neuron and how it could be simulated given input `X`, weights `W`, bias `b` and an activation function `f`. We saw how we could reduce the number of lines of code using vector operations and how by showing the weights and bias in a clever way could make the neuron calculate the function we wanted in a few cases. We also managed to connect multiple neurons and make the network calculate the `XOR` of 2 inputs, something not possible with a single neuron.

It was a bit of a hassle and quite a few lines of code with the small network and to verify that it did what we wanted. The goal in this notebook is to be able to calculate the output of a Artificial Neural Network in a easier way than before, and also to be able to verify how correct the output is. We will still do the maths ourselves, using vector operations and the numpy library.

In [1]:
import numpy as np

Assume we have an input layer with 4 inputs, a hidden layer with 3 neurons and an output layer with 2 neurons. We use `X` for the input values as before and `Y` for the output result. Ideally we would like to use `W` to represent the weights and `b` for the bias, but now we have two layers of neurons with multiple number of neurons in each layer, so we must make sure we don't confuse ourselves to much.

In this situation we could use the following variables:

```python
X = [x1, x2, x3, x4]  # the inputs
Wh1 = [?, ?, ?, ?]    # the weights of the first neuron in the hidden layer connected to all 4 inputs
Wh2 = [?, ?, ?, ?]    # the weights of the second neuron in the hidden layer connected to all 4 inputs
Wh3 = [?, ?, ?, ?]    # the weights of the third neuron in the hidden layer connected to all 4 inputs
Wo1 = [?, ?, ?]    # the weights of the first neuron in the output layer connected to all 3 hidden neurons
Wo2 = [?, ?, ?]    # the weights of the second neuron in the output layer connected to all 3 hidden neurons
bh1, bh2, bh3, bo1, bo2 = ? # the bias of our 5 neurons
```

To calculate the the output from the first neuron in the hidden layer we could do:

```python
zh1 = np.dot(X, W) + b
h1 = step(zh1)
```

It would be a bit laboursome to type this code for all the neurons though, and it wouldn't scale well. So, how can we do it more convenient, but still very clear. We could do similar to in the Neuron notebook:

```python
def g(X, W):
    z = np.dot(X, W)
    return step(z)
```

and we could create a function that evaluate the full network like this:

```python
def G(X, W1, W2, W3):
    h1 = g(X, W1)
    h2 = g(X, W2)
    y = g([h1, h2], W3)
    return y
```

Note that we did not need the bias in that example and therefore it is skipped in the above functions.

Lets try to create thos kind of functions for our current network. (By accident we forgot to consider the usage for this network, but we will ignore that for now and focuse on the structure and the code to simulate it.)

In [44]:
def step(z):
    return 1 if z > 0 else 0

def g(X, W, b):
    z = np.dot(X, W) + b
    return step(z)

def G(X, Wh1, Wh2, Wh3, bh1, bh2, bh3, Wo1, Wo2, bo1, bo2):
    h1 = g(X, Wh1, bh1)
    h2 = g(X, Wh2, bh2)
    h3 = g(X, Wh3, bh3)
    h = [h1, h2, h3]
    y1 = g(h, Wo1, bo1)
    y2 = g(h, Wo2, bo2)
    y = [y1, y2]
    return y

Hopefully `G` now correctly calculates the output of our 3 layer network (an input layer with 4 inputs, a hidden layer with 3 neurons and an output layer with 2 neurons), but yikes are there many parameters. Lets try to call the `G` and see what happens.

In [80]:
np.random.seed(1) # to have deterministic random values :-)
Wh1 = np.random.normal(size=4)
Wh2 = np.random.normal(size=4)
Wh3 = np.random.normal(size=4)
bh1 = np.random.normal()
bh2 = np.random.normal()
bh3 = np.random.normal() 
Wo1 = np.random.normal(size=3)
Wo2 = np.random.normal(size=3)
bo1 = np.random.normal() 
bo2 = np.random.normal()

X = [1, 0, 1, 0]
Y = G(X, Wh1, Wh2, Wh3, bh1, bh2, bh3, Wo1, Wo2, bo1, bo2)
print("X=" + str(X) + "  ->  Y=" + str(Y))
X = [1, 1, 1, 1]
Y = G(X, Wh1, Wh2, Wh3, bh1, bh2, bh3, Wo1, Wo2, bo1, bo2)
print("X=" + str(X) + "  ->  Y=" + str(Y))

X=[1, 0, 1, 0]  ->  Y=[0, 1]
X=[1, 1, 1, 1]  ->  Y=[1, 0]


It seemed to work, but all those parameters, and that handcrafted network function `G`, there must be a way to simplify that. Lets consider if we can group the neurons in the same layer together and calculate one layer at a time by using vectorisation and matrix multiplications. (We will skip the theoretical evidence part for now. :-)

To simplify things lets for a moment consider a network with 2 layers, one input layer with 2 inputs and one output layer with 2 neurons. This means we should be able to calculate the outputs like this:

```python
y1 = step(np.dot(X, W1) + b1)
y2 = step(np.dot(X, W2) + b2)
```

The question then becomes if we could combine the calculation of y1 and y2 in a way that scales in both readability and performance (there could be 100s of layers with 1000s of neurons or more in total).

Lets say we want this small network to implement the following truth table, to make sure we can verify our magic simplification we are about to do:

 x1 | x2 | y1 | y2
----|----|----|----
 0  | 0  | 0  | 0
 0  | 1  | 0  | 1
 1  | 0  | 0  | 1
 1  | 1  | 1  | 1
 
This means that neuron 1 is supposed to be an `AND` operation and neuron 2 an `OR` operation.

Lets first do it the old laboursome way:

In [122]:
def G(X):
    W1 = [1, 1]  # Lets hide the weights and bias inside our network function for this specifik network
    b1 = -1
    W2 = [1, 1]
    b2 = 0

    y1 = step(np.dot(X, W1) + b1)
    y2 = step(np.dot(X, W2) + b2)

    return [y1, y2]

In [125]:
examples = [
    ([0,0], [0,0]),
    ([0,1], [0,1]),
    ([1,0], [0,1]),
    ([1,1], [1,1])]

print([G(X) for X,_ in examples])  # Prints the outputs for all our four examples

print([G(X)==Y for X,Y in examples]) # Should output [True, True, True, True] if it matches our truth table

[[0, 0], [0, 1], [0, 1], [1, 1]]
[True, True, True, True]


Our network function now does what it is supposed to, and we have some magic code to verify that it is correct. Now it is time for some speculative refactoring and trying to use matrix multiplications.

```python
#Current version
def G(X):
    W1 = [1, 1]  # Lets hide the weights and bias inside our network function for this specifik network
    b1 = -1
    W2 = [1, 1]
    b2 = 0

    y1 = step(np.dot(X, W1) + b1)
    y2 = step(np.dot(X, W2) + b2)

    return [y1, y2]
```

In [203]:
# Potentially optimised version
def G(X):
    W1 = [1, 1]  # Lets hide the weights and bias inside our network function for this specifik network    
    W2 = [1, 1]
    b1 = -1
    b2 = 0
    
    W = np.array([W1,W2], ndmin=2).T
    B = np.array([b1,b2], ndmin=2)
    
    Z = np.dot(X, W) + B

    Y = [step(z) for z in Z]
    
    return Y[0] # to make sure the output is same type as it was before

print([G(X) for X,_ in examples])  # Prints the outputs for all our four examples

print([G(X)==Y for X,Y in examples]) # Should output [True, True, True, True] if it matches our truth table

[[0.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 1.0]]
[True, True, True, True]


The tests still holds, so it seems like the new version is actually implementing same functinality as the old version, but what is really happening? Lets take a look at the math here. 

Before we had one array with two weights per neuron, but know we combine them into a 2 x 2 matrix. Before we had:

```python
W1 = [w11, w12]  # The weights for neuron one
W2 = [w21, w22]  # The weights for neuron two
b1 = -1
b2 = 0
```

But now we combine them into:

```python
W = [
    [w11, w21],
    [w12, w22]
]
B = [b1, b2]
X = [x1, x2]
Z = np.dot(X, W) + B # = [z1, z2]
```

If all of this seems a bit confusing, don't worry to much, the intention is to learn a few thing about python and how operations with matrices can be used to simulate artifical neural networks (ANNs). The end goal is to have a library of functions that can be used to create and use ANNs and to know how to use them and for that it might help to know something about how they could be implemented.

So, lets go back and try to refactor that old network function for our initially planned network with 3 layers and 5 neurons. What we have since before is:

```python
def step(z):
    return 1 if z > 0 else 0

def g(X, W, b):
    z = np.dot(X, W) + b
    return step(z)

def G(X, Wh1, Wh2, Wh3, bh1, bh2, bh3, Wo1, Wo2, bo1, bo2):
    h1 = g(X, Wh1, bh1)
    h2 = g(X, Wh2, bh2)
    h3 = g(X, Wh3, bh3)
    h = [h1, h2, h3]
    y1 = g(h, Wo1, bo1)
    y2 = g(h, Wo2, bo2)
    y = [y1, y2]
    return y
```

Lets now try to redo it using matrix multiplications.

In [210]:
# First a version of the step function that also can be applied to an array, matrix or whatever we might want to throw at it
def step(z):
    if isinstance(z, (list, np.ndarray, tuple)):
        return [step(zi) for zi in z]
    else:
        return 1 if z > 0 else 0

In [223]:
# Now lets try the network function G
# X are the inputs as before
# L1 are the weights and bias for layer 1 as a tuple
# L2 are the weights and bias for layer 2 as a tuple
def G(X, L1, L2):
    W1, B1 = L1
    W2, B2 = L2
    
    H = step(np.dot(X, W1) + B1)  # Output from the hidden layer
    Y = step(np.dot(H, W2) + B2)  # Output from the network
       
    return Y

If the above code actualluy works, it is a major cleanup of the previous version, but how should we test it? Maybe us it to implement a binary adder:

x1 | x2 | y1 | y2
---|----|----|---
 0 |  0 |  0 |  0
 0 |  1 |  0 |  1
 1 |  0 |  0 |  1
 1 |  1 |  1 |  0
 
How should we break this down then so we can figure out the weights and bias? Well, looking at the truth table it seems like the following equations should work.

```
y1 = x1 and x2
y2 = (x1 or x2) and not (x1 and x2)
```

Can we fit this into the above network function `G` expecting 3 layers? It seems we can split the equations into:

```
h1 = x1 and x2
h2 = x1  or x2
y1 = h1
y2 = h2 and not h1
```

Feeling up to figuring out the needed weights and biases? Or do you long for some training algorithm to find them? ;-) For now we will handcode them.

In [225]:
L1 = ([[1,  1], 
       [1,  1]], 
      [-1,  0])
L2 = ([[1, -1], 
       [0,  1]], 
      [ 0,  0])

examples = [
    ([0, 0], [0, 0]),
    ([0, 1], [0, 1]),
    ([1, 0], [0, 1]),
    ([1, 1], [1, 0])
]

print([G(X, L1, L2) for X, _ in examples])
print([G(X, L1, L2) == Y for X, Y in examples])

[[0, 0], [0, 1], [0, 1], [1, 0]]
[True, True, True, True]


It seems like the output matches what we wanted. I don't know about you, but at least I need a break. If you actually understand why the selected weights and biases workes out in the code above, that is impressive, congratulate yourself. If you don't get it to add up, well, I had to fiddle abit to get it to work out, so don't worry to much about it. We will look into training a neetwork later. But first that break.